<a href="https://colab.research.google.com/github/shallex/QA-ruGPT3Small/blob/main/GPT3_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetune ruGPT3Small on QA Mail.ru

## Install enviroment

In [ ]:
!pip3 install folium==0.2.1

In [ ]:
!pip3 install urllib3==1.25.4 --upgrade

In [ ]:
!pip3 install transformers==2.8.0

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

In [ ]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
!sh setup.sh

## Add data to colab


In [ ]:
data_path = "/content/MAIL_TEXT.txt"
!ls "$data_path"

/content/MAIL_TEXT.txt


## Prepare data

In [ ]:
with open(data_path, "r", encoding = "ISO-8859-1") as file:
    text = file.read()

In [ ]:
topics = []
all_QA = []
for line in text.split("\n"):
    if "Q: " in line and "A: " in line:
        QA_text = line.split("A: ")
        if len(QA_text) == 2:
            qstn = QA_text[0].replace("Q: ", " ").strip()
            ans = QA_text[1].strip()
            QA_text = f"A: {ans}"
            QA_res = f"<s>Q: {qstn}\n{QA_text}</s>"
            all_QA.append(QA_res)
            topics.append(qstn)

In [ ]:
import numpy as np
import random

In [ ]:
random.seed(1234)
np.random.seed(1234)

In [ ]:
len(all_QA)

50497

In [ ]:
valid_size = 50000
print(valid_size)

50000


In [ ]:
unique_topics = list(set(topics))

In [ ]:
valid_topics = []

In [ ]:
idxs = set()
for _ in range(valid_size):
    idx = np.random.randint(0, len(unique_topics))
    idxs.add(idx)
    valid_topics.append(unique_topics[idx])

In [ ]:
import nltk


train = []
valid = []
c = 0
for idx, qa in enumerate(all_QA):
    is_train = True
    if idx in idxs:
        is_train = False
    if is_train:
        train.append(qa)
    else:
        valid.append(qa)
    c+=1
    if c%10000==0:
      print(c)

In [ ]:
len(train), len(valid)

In [ ]:
with open("train.txt", "w") as file:
    file.write("\n".join(train))

In [ ]:
with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

## Run finetuning


In [ ]:
!python pretrain_transformers.py \
    --output_dir=essays_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --fp16 \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

## Check model

In [ ]:
## Select topic
for idx in range(len(valid)):
  if "образует его общество" in valid[idx]:
    break

In [ ]:
valid[idx]

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=essays_model \
    --k=5 \
    --p=0.95 \
    --length=100